In [1]:
import os
import pandas as pd
from io import BytesIO
import numpy as np
from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm_notebook

In [20]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Reshape
from keras.layers import Average
from keras_contrib.layers import CRF
import keras
from sklearn.model_selection import train_test_split
# define documents
import numpy as np
import pandas as pd
import os
from tqdm.auto import tqdm
tqdm.pandas()
import pickle
# import matplotlib.pyplot as plt

In [2]:
from sklearn.utils import shuffle
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [4]:
dir_path = "/home/samarthgoal/language_vision/input/reddit/"

In [5]:
top_dir = "/home/samarthgoal/language_vision"
model_dir = os.path.join(top_dir,"models")

In [7]:
df_rage = pd.read_csv(os.path.join(dir_path,'processed_rage.csv'))
df_happy =  pd.read_csv(os.path.join(dir_path,'processed_happy.csv'))
df_gore =  pd.read_csv(os.path.join(dir_path,'processed_gore.csv'))
df_creepy =  pd.read_csv(os.path.join(dir_path,'processed_creepy.csv'))

In [8]:
# create a random balances dataset of all of the categories
length = np.min([len(df_rage),len(df_happy),len(df_creepy),len(df_gore)])

In [48]:
## combing all of the dataset
df_final = pd.concat([df_rage, df_happy, df_gore, df_creepy], ignore_index=True)

## load a filtred df (one that doesn't have wrong images)

In [46]:
## load the filtered df_
df_input = pd.read_csv(os.path.join(top_dir,"input","input_data.csv"))

## image preprocessing

In [61]:
import urllib.request
import os
import pandas as pd
from io import BytesIO
import numpy as np
from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
import pickle
from keras.layers import Lambda, Input
from keras.models import Model
from keras.backend import tf as ktf
import urllib.request
from requests.exceptions import ConnectionError
from tqdm import tqdm_notebook as tqdm

In [62]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as preprocess_input_incept
from keras.layers import MaxPooling1D
from keras.layers import Maximum
from keras.layers import Concatenate
from keras.layers import Dense
from keras.layers import Flatten
import numpy as np
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import urllib3

In [63]:
## trying with img1
# @ param: img url @ return feature vector 
from PIL import Image
import requests

In [64]:
def get_img_data(url):
    try : 
        response = requests.get(url)
        img_data = 0
        if (response.status_code == 200):
            try:
                if (url.endswith('.jpg') or url.endswith('.jpeg') or url.endswith('.png')):
                    with urllib.request.urlopen(url) as url1:
                        with open('temp.jpg', 'wb') as f:
                            f.write(url1.read())
                    fh = open('temp.jpg')
                    if fh:
                        img = image.load_img('temp.jpg', target_size= (299,299,3)) # 224*224
                        os.remove('temp.jpg')
                        img_data = image.img_to_array(img)
                    return img_data
            except RuntimeError as e:
                return 0  # except runtime error fro file that doesn't exits
        return img_data
    except IOError:
        return 0
    except FileNotFoundError as e:
        return 0
    except ConnectionError as e:
        return 0
    except urllib.error.HTTPError as e:
        return 0

In [ ]:
img_data = df_final['url'].progress_apply(get_img_data)

In [16]:
## load the image data
# img_data = pickle.load(open(os.path.join(model_dir,"img_data.pkl"),"rb"))

In [ ]:
## filter the data that doesn't have images
indices = ([idx for idx,ele in enumerate(img_data) if np.size(ele) == 1])
print(len(indices))
df_final = df_final.loc[~df_final.index.isin(indices)]

In [42]:
# create a random balances dataset of all of the categories
Y_new = df_final['subreddit']
# encode the output
Y_new = le.fit_transform(Y_new) 


In [43]:
## train test slplit
ids = list(df_final.id)
train_ids,test_ids,Y_train,Y_test = train_test_split(ids,Y_new,test_size=0.20,random_state=6)

In [44]:
## get the training imgae data
indices = df_final.index[df_final.id.isin(train_ids) == True].tolist()
img_data_train = np.array([img for idx,img in enumerate(img_data) if idx in indices])

In [24]:
img_data_train = preprocess_input_incept(img_data_train)

In [25]:
img_data_train.shape

(1178, 224, 224, 3)

In [ ]:
model = InceptionV3(weights='imagenet', include_top=True)

In [29]:
model_incept = Model(input=model.input, output=model.get_layer('avg_pool').output)

/home/samarthgoal/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  """Entry point for launching an IPython kernel.


In [30]:
model_incept.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 149, 149, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 149, 149, 32) 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [32]:
## get the prediction from a pre trained Inception net
img_data_train.shape

(1178, 224, 224, 3)

## Final dumping ground

In [ ]:
len(df_final)

In [45]:
## dump the final filtered data
df_final.to_csv(os.path.join(top_dir,"input","input_data.csv"))